In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import requests
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
load_dotenv()  # take environment variables from .env.

True

In [21]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("Top news in india today")

e:\Machine Learning and Data Science\LangChains\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [14]:
@tool
def get_weather_data(city: str) -> str:
    """ This function fetches the current weather data for a given city. """

    url = f"https://api.weatherstack.com/current?access_key=d4c55bf20603a678d7b7d56c5d347789&query={city}"

    response = requests.get(url)

    return response.json()

In [22]:
get_weather_data("hazaribagh")

{'request': {'type': 'City',
  'query': 'Hazaribagh, India',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Hazaribagh',
  'country': 'India',
  'region': 'Jharkhand',
  'lat': '23.983',
  'lon': '85.350',
  'timezone_id': 'Asia/Kolkata',
  'localtime': '2025-09-03 14:45',
  'localtime_epoch': 1756910700,
  'utc_offset': '5.50'},
 'current': {'observation_time': '09:15 AM',
  'temperature': 27,
  'weather_code': 143,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
  'weather_descriptions': ['Mist'],
  'astro': {'sunrise': '05:31 AM',
   'sunset': '06:05 PM',
   'moonrise': '03:05 PM',
   'moonset': '12:45 AM',
   'moon_phase': 'Waxing Gibbous',
   'moon_illumination': 75},
  'air_quality': {'co': '270.1',
   'no2': '7.585',
   'o3': '59',
   'so2': '7.215',
   'pm2_5': '8.14',
   'pm10': '8.325',
   'us-epa-index': '1',
   'gb-defra-index': '1'},
  'wind_speed': 24,
  'wind_degree': 89,
  'wind_dir': 'E',
  'pressure

In [4]:
results

"Check today's top 10 news stories, headline news, breaking news, latest news, politics news, sports news, entertainment news and business news on Times of India 3 days ago · Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at NDTV.com. 4 days ago · Today's news: Get latest and Breaking News on Politics, Business, Lifestyle, Entertainment and Sports along with News updates from around the world. Also, find English … 5 days ago · News Headlines Today: Hindustan Times offers the most reliable news in politics, sports, entertainment and business, featuring National and International news Breaking News on … National news and headlines from within or around India with live updates on major breaking events from The Hindu."

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [7]:
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--d14d7cc7-83ed-480f-b125-ef92b457e2e4-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [8]:
from langchain.agents import create_react_agent,AgentExecutor
from langchain import hub

In [9]:
# step 2 pull the react prompt from hub

prompt = hub.pull("hwchase17/react")

e:\Machine Learning and Data Science\LangChains\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
# step 3 create the react agent manually with the pulled prompt 

agent = create_react_agent(
    llm=llm,
    tools=[search_tool,get_weather_data],
    prompt = prompt
)

In [24]:
# step4 wrap it with AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool,get_weather_data],
    verbose=True
)

In [25]:
# step 5 invoke

response = agent_executor.invoke({"input":"Find the capital of Madhya Pradesh, then find it's current weather conditions"})
print(response)



> Entering new AgentExecutor chain...
I need to first find the capital of Madhya Pradesh. Then, I need to find the current weather conditions in that city.
Action: duckduckgo_search
Action Input: capital of Madhya Pradesh

e:\Machine Learning and Data Science\LangChains\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Bhopal is the capital of Madhya Pradesh. Bhopal is the administrative headquarters. This article will shed more light on the capital city of Bhopal and the state of Madhya Pradesh. Bhopal Bhopal is … The capital city of the state Madhya Pradesh is Bhopal. The city is known for its various natural and artificial lakes and for being one of the greenest cities in India. Aug 13, 2023 · Answer: Indore is not the capital of any Indian state. Indore is the largest city in the Indian state of Madhya Pradesh, but the capital of Madhya Pradesh is Bhopal. Bhopal serves as … Mar 11, 2023 · Indore has the largest economy in Madhya Pradesh and is the business and trading capital of the state. Pithampur Industrial Area, near Indore houses more than 1200 industrial units. Jun 16, 2024 · What is the capital of Madhya pradesh ? Get the answers you need, now!I now know the capital is Bhopal. I need to find the weather conditions for Bhopal.
Action: get_weather_data
Action Input: Bhopal{'request': {'type'

In [26]:
response['output']

'The capital of Madhya Pradesh is Bhopal. The current weather in Bhopal is light rain with a temperature of 27 degrees Celsius. The wind speed is 14 km/h.'